In [1]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer
import tensorflow as tf

# Loading the GPT-2 model in tensorflow

In [2]:
model = TFAutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


# Data Preparation

In [4]:
import pandas as pd
df = pd.read_csv('Furniture_Dataset.csv')
df.columns = df.columns.str.strip()
print(df.head())
data = []
for index, row in df.iterrows():
    input_text = f"Calculate shipping cost for {row['Product']} with dimensions {row['Length']}x{row['Breadth']}x{row['Height']} cm and weight {row['Weight']} kg."
    output_text = "Shipping cost is $50"
    data.append({"input": input_text, "label" : output_text})

transformed_df = pd.DataFrame(data)
transformed_df.to_csv('training_data.csv', index = False)

             Product  Length  Breadth  Height  Weight
0              Chair      20       20      30       5
1              Table      60       40      30      15
2     BILLY Bookcase      80       28     202      25
3     MALM Bed Frame     209      177      38      32
4  LACK Coffee Table     118       78      45      12


# Tokenization and Data Modeling

In [5]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer, create_optimizer
import tensorflow as tf
import pandas as pd

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = TFAutoModelForCausalLM.from_pretrained("gpt2")

# Ensure tokenizer has a pad token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.convert_tokens_to_ids('[PAD]')

# Load and preprocess the training data
df = pd.read_csv('training_data.csv')
inputs = df['input'].tolist()
labels = df['label'].tolist()

# Tokenize inputs and labels
max_length = 512  # Adjust max_length as needed
tokenized_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
tokenized_labels = tokenizer(labels, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")

# Convert tensors to a format compatible with TensorFlow Dataset
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']
labels = tokenized_labels['input_ids']

# Create TensorFlow dataset with reduced batch size
def create_tf_dataset(input_ids, attention_mask, labels, batch_size=4):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }, labels
    ))
    
    # Calculate maximum sequence length for padding
    max_length = input_ids.shape[1]

    # Pad the batches to ensure consistent shapes
    dataset = dataset.padded_batch(
        batch_size,
        padded_shapes=({
            'input_ids': [max_length],
            'attention_mask': [max_length]
        }, [max_length]),
        padding_values=({
            'input_ids': tokenizer.pad_token_id,
            'attention_mask': 0
        }, tokenizer.pad_token_id)
    )
    
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = create_tf_dataset(input_ids, attention_mask, labels, batch_size=4)

# Define a custom loss function
def custom_loss(y_true, y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

# Create a Keras optimizer
optimizer, _ = create_optimizer(init_lr=5e-5, num_train_steps=len(df)//4, num_warmup_steps=100)

# Enable mixed precision training
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Compile the model with the optimizer and custom loss function
model.compile(optimizer=optimizer, loss=custom_loss)

# Train the model
model.fit(train_dataset, epochs=3)

# Save the model and tokenizer
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Using pad_token, but it is not set yet.


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Epoch 1/3
6/6 [==============================] - 350s 49s/step - loss: 8.4849
Epoch 2/3
6/6 [==============================] - 271s 44s/step - loss: 7.0560
Epoch 3/3
6/6 [==============================] - 284s 46s/step - loss: 4.2484


('./fine_tuned_gpt2\\tokenizer_config.json',
 './fine_tuned_gpt2\\special_tokens_map.json',
 './fine_tuned_gpt2\\vocab.json',
 './fine_tuned_gpt2\\merges.txt',
 './fine_tuned_gpt2\\added_tokens.json',
 './fine_tuned_gpt2\\tokenizer.json')

In [6]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM

# Load the pre-trained model and tokenizer
model = TFAutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Save the model and tokenizer
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


('./fine_tuned_gpt2\\tokenizer_config.json',
 './fine_tuned_gpt2\\special_tokens_map.json',
 './fine_tuned_gpt2\\vocab.json',
 './fine_tuned_gpt2\\merges.txt',
 './fine_tuned_gpt2\\added_tokens.json',
 './fine_tuned_gpt2\\tokenizer.json')

In [7]:
tokenizer = AutoTokenizer.from_pretrained('./fine_tuned_gpt2')

In [11]:
def calculate_shipping_cost(length, breadth, height, weight, pincode):
    base_rate = 50
    metro_pincodes = ['1001', '1002', '2001']
    same_city_pincodes = ['1000']

    is_metro = pincode in metro_pincodes
    is_same_city = pincode in same_city_pincodes

    height_surcharge = 20 if height > 60 else 0
    weight_surcharge = 15 if weight > 8 else 0

    distance_factor = 0 if is_same_city else (30 if is_metro else 50)

    total_cost = base_rate + height_surcharge + weight_surcharge + distance_factor
    return total_cost


In [26]:
from flask import Flask, request, jsonify
from transformers import TFAutoModelForCausalLM, AutoTokenizer
import pandas as pd

app = Flask(__name__)

# Load the fine-tuned model and tokenizer
model = TFAutoModelForCausalLM.from_pretrained('./fine_tuned_gpt2')
tokenizer = AutoTokenizer.from_pretrained('./fine_tuned_gpt2')

def load_pincodes(file_path):
    with open(file_path, 'r') as file:
        return set(line.strip() for line in file)

def generate_shipping_cost(query):
    input_ids = tokenizer.encode(query, return_tensors='tf')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

metro_pincodes = load_pincodes('metro_pincodes.txt')
same_city_pincodes = load_pincodes('same_city_pincodes.txt')
undeliverable_pincodes = ['110080','110090','700104','700112','400104','600098','412404','412407']

@app.route('/')
def home():
    return "Welcome to the Shipping Cost Calculator API!"

@app.route('/get_shipping_cost', methods=['POST'])
def get_shipping_cost():
    try:
        data = request.get_json()
#         print("Received data :", data)
        product_name = data.get('product_name')
#         print("Product_name : ", product_name)
        pincode = data.get('pincode')
#         print("pincode : ", pincode)
        
        # Load product details
        df = pd.read_csv('furniture_dataset.csv')
        product = df[df['Product'] == product_name]

        if product.empty:
            return jsonify({'error': 'Product not found'}), 404
        
        # Check if the pincode is deliverable
        if pincode in undeliverable_pincodes:
            return jsonify({'error':'Delivery is not available to this pincode'}), 400
        
        # Calculate shipping cost
        dimensions = product[['Length', 'Breadth', 'Height']].values[0]
        weight = product['Weight'].values[0]

        # Sample cost calculation logic
        base_shipping_cost = 50  # Base cost
        additional_cost = 0

        # Add extra cost for oversized items
        if dimensions[2] > 2 * 30.48 or weight > 8:  # 2 feet in centimeters
            additional_cost += 20  # Additional charge for oversized items

        # Check if the pincode is in a metro city
        if pincode in metro_pincodes:
            shipping_cost = base_shipping_cost  # Lower cost for metro cities
        else:
            shipping_cost = base_shipping_cost + 30  # Higher charge for non-metro cities

        total_cost = shipping_cost + additional_cost

        return jsonify({'shipping_cost': total_cost})
    
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    try:
        app.run()
    except KeyboardInterrupt:
        sys.exit()


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./fine_tuned_gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Sep/2024 01:27:53] "POST /get_shipping_cost HTTP/1.1" 400 -


In [28]:
import requests

def get_shipping_cost_from_flask(product_name, pincode):
    url = 'http://127.0.0.1:5000/get_shipping_cost'
    payload = {'product_name': product_name, 'pincode': pincode}
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        return response.json().get('shipping_cost')
    else:
        return response.json().get('error')

def chat():
    while True:
        print("Enter 'exit' to quit.")
        print("Enter product name:")
        product_name = input()
        
        if product_name.lower() == 'exit':
            break
        
        print("Enter pincode:")
        pincode = input()
        
        if pincode.lower() == 'exit':
            break

        shipping_cost = get_shipping_cost_from_flask(product_name, pincode)
        
        if isinstance(shipping_cost, str):  # Error message
            print(f"Error: {shipping_cost}")
        else:
            print(f"The shipping cost is: ${shipping_cost}")

if __name__ == '__main__':
    chat()


Enter 'exit' to quit.
Enter product name:
chair
Enter pincode:
411043
Error: Product not found
Enter 'exit' to quit.
Enter product name:
exit
